# SILO Patch Point Dataset

SILO is an enhanced climate database hosted by the Science Delivery Division of the Department of Science, Information Technology and Innovation (DSITI). SILO contains Australian climate data from 1889 (current to yesterday), suitable for research and climate applications. More information is available at the [SILO Climate Data](https://www.longpaddock.qld.gov.au/silo/) website.

All SILO climate data has been provided to UQ, and has been downloaded and is available on the Master of Data Science Datasets [website](https://mdatascience.uqcloud.net/SILO_PPD/) in the form of space-deliminated files with a daily time series of data at point locations consisting of station records which have been supplemented by interpolated estimates when observed data are missing. Patched datasets are available at approximately 4800 Bureau of Meteorology recording stations around Australia.

**The datasets provided are only to be used for learning purposes as specified in the respective course(s). These datasets are available publicly (unless otherwise stated), use of these publicly available datasets is strictly limited to learning purposes in the master of data science course. If you wish to use these datasets for any other purpose please access them directly from the source. UQ accepts no responsibility for any use of these datasets outside of these conditions.**

## Proposed data-science problems

SILO has suggested a number of problems that might be worth looking into with this dataset:

### Error detection in climate data

The project would aim to develop an error detection system for identifying erroneous values in climate datasets. The proposed system could use a range of algorithms, from simple range checking (to ensure data values are within an acceptable range), to advanced machine learning or dynamic climate modelling methods. The analysis could examine previous observations from the same station records (temporal analysis) and/or station records from nearby stations (spatial analysis).  Other sources of data verification could be used if they are readily available (for example, weather forecasts, satellite estimates, etc.). Students are encouraged to propose any viable method even if it doesn’t result in usable software. Participating students are expected to outlinethe proposed methodology and plan for evaluation.

### Detecting trends in climate datasets

The project would aims to identify systematic trends or biases in the “Patched Point Datasets (PPDs)” provided by SILO. A “patched” dataset is a daily time series of climate data at a given location, consisting of observed data for a given variable (for example, rainfall) when observed data are available, and interpolated data when observed data are missing. The goal is to develop an algorithm which can determine if there is a significant difference between observed and in-filled (interpolated) data. The determination should be made by computing statistical parameters for period(s) when observed data are provided, and for period(s) when interpolated estimates are provided.  Ambitious students may also wish to investigate methods for adjusting the interpolated data so it has statistical properties (eg. mean, variance) matching the observed data.

### Predicting future values and/or missing stations

Can you use this dataset to either predict future values at an existing station, predict the values of a missing station at a particularly geospation location or predict the value of a particular weather observation in time from other observations (such as temperature from insolation). Either of these approaches could be investigated by keeping appropriate data separate, building models, and comparing the output of those models to the left-out data.

## Getting The Data I Need

The SILO data is provided as around 4800 space-deliminated files, with the list of files provided in the file [all_files.txt](https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/all_files.txt) You can also browse the files manually using the data science browser at: https://mdatascience.uqcloud.net/SILO_PPD/

As the files account to almost 200GB, they have also been placed into a shared HDFS directory under `/shared/silo` and imported into Hive for further exploration.


## Is my data fit for use?
The first thing you may notice is that there are many individual files, totalling about 200GB. This amount of data is far larger than what can fit into memory on a single machine. However, the individual files alone are small enough to fit in memory and analyse using traditional tools. One of the largest challenge of big data problems is deciding when using a cluster might be appropriate and when using our traditional scripting tools such as Python or R might be.

In our case, we can use Hive/distributed processing to ask questions about the *entire dataset*, which may guide us into what files are worth looking at for closer inspection. Since submitting a job over a large dataset can take a lot of time, it's worth examining individual files first to determine what sort of questions you want answered about the entire dataset.

Let's first see how many files there are, we will use Python to scrape some information from the data repository.

In [ ]:
import requests
from IPython.display import display
r = requests.get('https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/all_files.txt')
all_files = [f.strip() for f in r.text.split('\n') if f.strip()]
print("There are {} PPD files available".format(len(all_files)))
print("First 10 files:")
display(all_files[0:10])

### Getting file metadata
Each file has a very large header in the CSV before it gets down to the actual data:

In [ ]:
def get_file_lines(filename):
    r = requests.get('https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/{}'
                   .format(filename))
    return [l.strip() for l in r.text.split('\n')]
print '\n'.join(get_file_lines('3006_UQ.dat')[0:55])

This header is identical for all files, with the exception of the lines

```" * Patched Point data for station:  3006 FITZROY CROSSING COMP.                   Lat: -18.1919 Long: 125.5644"
" * Elevation:  114m "```

So, let's see if we can get that extracted for all files (and save it for use later).

In [ ]:
import re
def get_metadata(filename):
    metadata={}
    r = requests.get('https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/{}'
                   .format(filename))
    for line in r.text.split('\n'):
        # Use regular expressions to pattern match
        # Effectively, we want to match the string after "station:" and the floats after
        # "Lat: " and "Long: "
        PPD_match = re.search('station: *(?P<id>[^ ]+)' +
                              ' (?P<name>.*) ' +
                              'Lat: (?P<latitude>[0-9.-]+) ' +
                              'Long: (?P<longitude>[0-9/.-]+)', line)
        if PPD_match:
            metadata['id'] = PPD_match.group('id')
            metadata['name'] = PPD_match.group('name').strip()
            metadata['latitude'] = float(PPD_match.group('latitude'))
            metadata['longitude'] = float(PPD_match.group('longitude'))
        else:
            elevation_match = re.search('Elevation: *(?P<elevation>[0-9]+)m', line)
            if elevation_match:
                metadata['elevation'] = int(elevation_match.group('elevation'))
                # once we have the elevation, all metadata is collected, and we can stop
                break
            
    return metadata
            
get_metadata('3006_UQ.dat')

Looks like it worked. Let's try it on all files.

In [ ]:
import pandas as pd
all_metadata = pd.read_csv('all_metadata.csv')
for i in range(all_metadata.shape[0]):
    if pd.isnull(all_metadata.loc[i,'name']):
        filename = all_metadata.loc[i,'filename']
        print("Extracting metadata from {} ({}/{})".format(filename, i+1, len(all_files)))
        for k, v in get_metadata(filename).items():
            all_metadata.loc[i,k] = v

In [ ]:
all_metadata.tail()

Now that we've extracted the metadata from the files, let's save this to disk as a new csv called `all_metadata.csv` we can use for future analysis.

In [ ]:
all_metadata.to_csv('all_metadata.csv', index=False)

### Metadata

Now that we have parsed the metadata out of the raw data, let's use it to visualise some of the data. Since we have latitutde and longitude, we can the location of each data point on a map:

In [ ]:
from matplotlib import pyplot as plt
plt.plot(all_metadata['longitude'],all_metadata['latitude'],'r,')
plt.title("Weather station locations")
plt.xlabel('Longitude')
plt.ylabel('Latitude')

It seems like most of the weather station locations are centred around major city areas.

### Weather-station data

Let's choose a station at random, and see what sort of data we can retrieve. According to the documentation we should get the following fields:

|Field|Type|Description|
| ---- | 
| date | string (yyyymmdd) | date of observation |
| day | integer | day of year |
| date2 | string (dd-mm-yyyy) | date of observation (just different format) |
| T.Max | float | maximum temperature reached (Celsius) |  
| Smx | integer | source of maximum temperature (see Sources) |
| T.Min | float | minimum temperature reached (Celsius) |
| Smn | integer | source of minimum temperature (see Sources) |
| Rain | float | millimetres of rainfall |
| Srn | integer | source of rainfall (see Sources) |
| Evap | float | millimetres of evaporation | 
| Sev | integer | source of evaporation (see Sources) |
| Radn | float | radiation (in MJ/m^2) |
| Ssl | integer | source of radiation (see Sources) |
| VP | float | vapour pressure (in hPa) |
| Svp | integer | source of vapour pressure |
| RHmaxT | float | estimated relative humidity at T.Max (percent 0-100) |
| RHminT | float | estimated relative humidity at T.Min (percent 0-100) |
| FAO56 | float | [Short-crop potential evapotranspiration](http://www.fao.org/docrep/X0490E/X0490E00.htm) (mm) |
| ASCEPM | float | [ASCE tall-crop potential evapotranspiration](http://www.kimberly.uidaho.edu/water/asceewri/ASCE_Standardized_Ref_ET_Eqn_Phoenix2000.pdf) (mm) |
| Mlake | float | Morton evaporation over shallow lakes (mm) |
| Mpot | float | Morton potential evapotranspiration over land (mm) |
| Mact | float | Morton actual evapotranspiration over land (mm) |
| Mwet | float | Morton wet environment areal evapotranspiration over land (mm) |
| Span | float | calibrated estimate of class A pan evaporation (mm) |
| Ssp | integer | source of pan evaporation (see Sources) |
| EvSp | float | class A evaporation (used post 1970) and synthetic pan evaporation (pre 1970) (mm) |
| MSLPres | float | mean sea level pressure (hPa) |
| Sp | integer | source of MSLPres (see Sources) |

#### Sources

Many of the above fields have a corresponding 'source' field, which is an integer code corresponding to the following sources:

| source code | description
| ---------- |
| 0 | Station data, as supplied by Bureau |
| 13 | Deaccumulated using nearby station |
| 15 | Deaccumulated using interpolated data |
| 23 | Nearby station, data from BoM | 
| 25 | interpolated daily observations |
| 26 | synthetic pan evaporation (only used for Ssp) |
| 35 | interpolated from daily observations using anomaly interpolation method for CLIMARC data | 
| 75 | interpolated long term average | 

Let's put those into a variable, for use later:

In [ ]:
source_names = {0: 'station',
               13: 'deaccum-nearby',
               15: 'deaccum-interp',
               23: 'nearby-BoM',
               25: 'interp-daily',
               26: 'synth-pan',
               35: 'interp-daily-CLIMARC',
               75: 'interp-lta'}

Let's load up a station, and have a look at its data:

In [ ]:
%matplotlib inline
import random

def choose_station():
    choice_index = random.randint(0,len(all_metadata)-1)
    return all_metadata.iloc[choice_index,:]

choice = all_metadata.iloc[1,:]
# choice = choose_station()
print choice

# basemap
plt.figure(figsize=[10,10])
plt.plot(all_metadata['longitude'],all_metadata['latitude'],'r,')
plt.title("Weather station locations")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
# chosen weather station
x, y = [choice['longitude'], choice['latitude']]
plt.plot(x, y, 'b.')
plt.annotate('{} [{}]'.format(choice['name'], choice['filename']), xy=[x, y], xytext=[x, y+0.3], color='blue')

Ok, let's grab that file, and see what we've got:

We learnt from the metadata extraction script that the first 53 lines are various documentation and can be ignored. line 54 is the headers, but line 55 is units and can be ignore, at least for data input. We should be able to use `header=53` (zero-indexed) and `skiprows=[54]` to do this, but something is weird in Pandas, and actually, `header=47` and `skiprows=[48]` appears to work, so we'll move on for now.

In [ ]:
def load_data(choice):
    url = 'https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/' + choice['filename']
    print ("Loading {}".format(url))
    data = pd.read_csv(url, header=47, skiprows=[48], skip_blank_lines=False, 
                       delim_whitespace=True, parse_dates=[2], dayfirst=True)
    return data

data = load_data(choice)
data.head()

Let's have a look at the temperatures over time:

In [ ]:
plt.figure(figsize=[15,5])
plt.plot(data['Date2'],data['T.Max'],'r-')
plt.plot(data['Date2'],data['T.Min'],'g-')
plt.legend(['T.Max','T.min'])

Let's smooth the data and see if we can get a clearer long term picture.

In [ ]:
# we should smooth that and see if we can get a clearer long-term picture
TMax_smoothed = data['T.Max'].rolling(window=30).mean()
TMin_smoothed = data['T.Min'].rolling(window=30).mean()
plt.figure(figsize=[15,5])
plt.plot(data['Date2'],data['T.Max'],'r-', alpha=0.1)
plt.plot(data['Date2'],TMax_smoothed,'r-')
plt.plot(data['Date2'],data['T.Min'],'g-', alpha=0.1)
plt.plot(data['Date2'],TMin_smoothed,'g-')

From this specific station, it doesn't look like there are any obvious patterns of temperature changes over time, except from around 1961 where the pattern seems to change. Why is this? You may recall from earlier that the data contains a variety of different *sources*. Some data is measured, and some is interpolated. Part of determining if our data is fit for use is to make sure that the data is accurate, which may involve determining examining the sources. Let's see where this data is sourced from.

In [ ]:
# let's look at the sources of TMax and TMin for this station
import numpy as np
indexes = np.arange(0, len(source_names))
fig, ax = plt.subplots(figsize=(15,5))
width=0.35
h1 = ax.bar(indexes,
        [sum(data['Smx'] == k) for k in source_names.keys()],
      color='r', width=width)
h2 = ax.bar(indexes + width,
        [sum(data['Smn'] == k) for k in source_names.keys()],
      color='g', width=width)
ax.set_xticks(indexes + width/2)
ax.set_xticklabels(source_names.values());
ax.legend((h1[0], h2[0]), ('T.Max', 'T.Min'))
ax.set_title('Source of Temperature Data for {}'.format(choice['name']))

It looks like much of the data is interpolated, rather than measured, using two different methods of interpolation. Maybe we can look at these sources over time to determine when each method of interpolation was used?

In [ ]:
import matplotlib.patches as mpatches

# create a function, so we can call it multiple times
def plot_data_by_source(data, val_field, source_field, name, date_field='Date2', source_names=source_names):

    # helper function to ensure we stay within the size of colors
    def get_color(i):
        colors = plt.get_cmap('tab10').colors
        return colors[i % len(colors)]

    plt.figure(figsize=[15,5])

    handles = []

    for i, s in enumerate(source_names.keys()):
        
        select = (data[source_field] == s)
        if sum(select) > 0:
            plt.plot(data[date_field][select],data[val_field][select],'.', color=get_color(i))
            # create handle for legend
            handles.append(mpatches.Patch(color=get_color(i), 
                                          label='{} ({:0.2f}%)'.format(
                                              source_names[s], 
                                              100.0*sum(select)/len(select))))
    
    # legend
    plt.legend(handles=handles)
    plt.title('Source of {} data for {}'.format(val_field, name))

In [ ]:
plot_data_by_source(data, 'T.Max', 'Smx', '{} [{}]'.format(choice['name'], choice['filename']))

It looks like in 1961 the source changed, and actual data was used rather than interpolated data. This explains the shift in temperatures we saw during that time period.

Let's pick another *random* station and see if the source of the data also uses the same methods of interpolations. Make sure you run the following code (with shift+enter on the block):

In [ ]:
# now lets look at another
new_choice = choose_station()
plot_data_by_source(load_data(new_choice), 'T.Max', 'Smx', 
                    '{} [{}]'.format(new_choice['name'], new_choice['filename']))

How does this differ? Are the sources of this data different? Remember, since the dataset you picked is *random*, each student will have different silo data. Try running the following blocks of code again to continue to pick random locations, and get a feel of how the sources change depending on location. 

In [ ]:
# now lets look at another
new_choice = choose_station()
plot_data_by_source(load_data(new_choice), 'T.Max', 'Smx', 
                    '{} [{}]'.format(new_choice['name'], new_choice['filename']))

The difference between sources suggests that some datasets might be more trustworthy than others. When determining if our data is fit for use, or what dataset to focus on, we will have to also take into account *how* that data was obtained, as it will change how we should analyse it.

Given that there are 4000 files, this technique of looking at each one manually to find trustworthy datasets to further analyse on a single computer would be time consuming. In the next session, we will use our hadoop cluster to perform distributed analysis to get an understanding of the data *as a whole* rather than individual files, and use that knowledge to select a few representative files to analyse further.